In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv(dynamic_pricing.csv')
df.head()

In [ ]:
# Criar um dicionário de tradução
colunas = {
    'Number_of_Riders': 'Número_de_Passageiros',
    'Number_of_Drivers': 'Número_de_Motoristas',
    'Location_Category': 'Categoria_de_Localização',
    'Customer_Loyalty_Status': 'Status_de_Lealdade_do_Cliente',
    'Number_of_Past_Rides': 'Número_de_Viagens_Anteriores',
    'Average_Ratings': 'Avaliações_Médias',
    'Time_of_Booking': 'Horário_da_Reserva',
    'Vehicle_Type': 'Tipo_de_Veículo',
    'Expected_Ride_Duration': 'Duração_Estimada_da_Viagem',
    'Historical_Cost_of_Ride': 'Custo_Histórico_da_Viagem'
}

# Renomear as colunas inplace
df.rename(columns=colunas, inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Inicializar listas para armazenar as colunas categóricas e numéricas
colunas_categoricas = []
colunas_numericas = []

# Iterar sobre as colunas do DataFrame
for coluna in df.columns:
    if pd.api.types.is_categorical_dtype(df[coluna]):
        colunas_categoricas.append(coluna)
    elif pd.api.types.is_numeric_dtype(df[coluna]):
        colunas_numericas.append(coluna)
        


In [ ]:
# Criar DataFrame para colunas categóricas
df_categorico = df.loc[:, colunas_categoricas]

# Criar DataFrame para colunas numéricas
df_numerico = df.loc[:, colunas_numericas]

In [ ]:
df_numerico

In [ ]:
df_categorico

In [ ]:
mediana = df_numerico.median()
mediana

In [ ]:
# Selecionar apenas as colunas numéricas
df_numericas = df[colunas_numericas]

# Calcular a matriz de correlação
matriz_correlacao = df_numericas.corr()
matriz_correlacao 

In [ ]:
# Calcular a matriz de correlação
correlation_matrix = df_numericas.corr()

# Configurar o tamanho da figura
plt.figure(figsize=(10, 8))

# Plotar o gráfico de calor da matriz de correlação
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

# Adicionar título ao gráfico
plt.title('Matriz de Correlação')

# Mostrar o gráfico
plt.show()

In [ ]:
# Gráfico de dispersão para avaliações médias em relação ao número de passageiros
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Número_de_Passageiros', y='Avaliações_Médias')
plt.title('Gráfico de Dispersão: Avaliações Médias vs. Número de Passageiros')
plt.xlabel('Número de Passageiros')
plt.ylabel('Avaliações Médias')
plt.show()

# Boxplot para avaliações médias em relação ao número de viagens anteriores
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='Número_de_Viagens_Anteriores', y='Avaliações_Médias')
plt.title('Boxplot: Avaliações Médias por Número de Viagens Anteriores')
plt.xlabel('Número de Viagens Anteriores')
plt.ylabel('Avaliações Médias')
plt.show()

In [ ]:
# Configurar o tamanho da figura
plt.figure(figsize=(12, 5))

# Plotar o histograma da duração estimada da viagem
plt.subplot(1, 2, 1)
sns.histplot(df['Duração_Estimada_da_Viagem'], bins=20, kde=True, color='skyblue')
plt.title('Distribuição da Duração Estimada da Viagem')
plt.xlabel('Duração Estimada da Viagem')

# Plotar o gráfico de densidade do custo histórico da viagem
plt.subplot(1, 2, 2)
sns.kdeplot(df['Custo_Histórico_da_Viagem'], shade=True, color='salmon')
plt.title('Distribuição do Custo Histórico da Viagem')
plt.xlabel('Custo Histórico da Viagem')

# Ajustar o layout
plt.tight_layout()

# Mostrar os gráficos
plt.show()

In [ ]:
# Configurar o tamanho da figura
plt.figure(figsize=(8, 6))

# Plotar o gráfico de dispersão da duração estimada da viagem vs. custo histórico da viagem
sns.scatterplot(data=df, x='Duração_Estimada_da_Viagem', y='Custo_Histórico_da_Viagem', color='blue', alpha=0.5)
plt.title('Duração Estimada da Viagem vs. Custo Histórico da Viagem')
plt.xlabel('Duração Estimada da Viagem')
plt.ylabel('Custo Histórico da Viagem')

# Mostrar o gráfico
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.columns

In [ ]:
# Percentis para alta e baixa demanda
percentil_alta_demanda = 75
percentil_baixa_demanda = 25

# Calculando multiplicador de demanda
df['multiplicador_demanda'] = np.where(df['Número_de_Passageiros'] > np.percentile(df['Número_de_Passageiros'], percentil_alta_demanda),
                                        df['Número_de_Passageiros'] / np.percentile(df['Número_de_Passageiros'], percentil_alta_demanda),
                                        df['Número_de_Passageiros'] / np.percentile(df['Número_de_Passageiros'], percentil_baixa_demanda))

# Percentis para alta e baixa oferta
percentil_alta_oferta = 75
percentil_baixa_oferta = 25

# Calculando multiplicador de oferta
df['multiplicador_oferta'] = np.where(df['Número_de_Motoristas'] > np.percentile(df['Número_de_Motoristas'], percentil_baixa_oferta),
                                       np.percentile(df['Número_de_Motoristas'], percentil_alta_oferta) / df['Número_de_Motoristas'],
                                       np.percentile(df['Número_de_Motoristas'], percentil_baixa_oferta) / df['Número_de_Motoristas'])

# Fatores de ajuste de preço para alta e baixa demanda/oferta
limite_demanda_alta = 1.2  # Limite de demanda alta
limite_demanda_baixa = 0.8   # Limite de demanda baixa
limite_oferta_alta = 0.8  # Limite de oferta alta
limite_oferta_baixa = 1.2   # Limite de oferta baixa

# Calculando custo ajustado da viagem para precificação dinâmica
df['custo_viagem_ajustado'] = df['Custo_Histórico_da_Viagem'] * (
    np.maximum(df['multiplicador_demanda'], limite_demanda_baixa) *
    np.maximum(df['multiplicador_oferta'], limite_oferta_alta)
)

In [ ]:
import plotly.graph_objects as go

# Calcular a porcentagem de lucro
df['porcentagem_lucro'] = ((df['custo_viagem_ajustado'] - df['Custo_Histórico_da_Viagem']) / df['Custo_Histórico_da_Viagem']) * 100

# Identificar viagens lucrativas onde a porcentagem de lucro é positiva
viagens_lucrativas = df[df['porcentagem_lucro'] > 0]

# Identificar viagens com prejuízo onde a porcentagem de lucro é negativa
viagens_prejuizo = df[df['porcentagem_lucro'] < 0]

# Calcular o número de viagens lucrativas e com prejuízo
contagem_lucrativas = len(viagens_lucrativas)
contagem_prejuizo = len(viagens_prejuizo)

# Criar um gráfico de rosca para mostrar a distribuição de viagens lucrativas e com prejuízo
rótulos = ['Viagens Lucrativas', 'Viagens com Prejuízo']
valores = [contagem_lucrativas, contagem_prejuizo]

figura = go.Figure(data=[go.Pie(labels=rótulos, values=valores, hole=0.4)])
figura.update_layout(title='Rentabilidade das Viagens (Precificação Dinâmica vs. Precificação Histórica)')
figura.show()


In [ ]:
import plotly.express as px

# Gráfico de dispersão com linha de tendência (OLS)
fig = px.scatter(df, 
                 x='Duração_Estimada_da_Viagem', 
                 y='Custo_Histórico_da_Viagem',
                 title='Duração Estimada da Viagem vs. Custo Histórico da Viagem', 
                 trendline='ols')
fig.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Selecionar as características relevantes para o clustering
caracteristicas = ['Número_de_Viagens_Anteriores', 'Avaliações_Médias', 'Duração_Estimada_da_Viagem', 'Custo_Histórico_da_Viagem']

# Criar um DataFrame apenas com as características selecionadas
dados_cluster = df[caracteristicas]

# Padronizar os dados para que todas as características tenham a mesma escala
scaler = StandardScaler()
dados_cluster_normalizados = scaler.fit_transform(dados_cluster)

# Escolher o número de clusters (grupos de clientes) - você pode ajustar esse valor conforme necessário
n_clusters = 4

# Instanciar o modelo de K-means com o número de clusters desejado
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Aplicar o algoritmo de clustering aos dados
kmeans.fit(dados_cluster_normalizados)

# Adicionar os rótulos dos clusters aos dados originais
df['Cluster'] = kmeans.labels_

# Visualizar os clusters em relação às características
for feature in caracteristicas:
    plt.figure(figsize=(8, 5))
    for cluster in range(n_clusters):
        plt.hist(df[df['Cluster'] == cluster][feature], alpha=0.5, label=f'Cluster {cluster}')
    plt.title(f'Distribuição de {feature} por Cluster')
    plt.xlabel(feature)
    plt.ylabel('Frequência')
    plt.legend()
    plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# Selecionar todas as características, exceto a variável alvo (por exemplo, 'Custo_Histórico_da_Viagem')
X = df.drop('Custo_Histórico_da_Viagem', axis=1)

# Selecionar a variável alvo
y = df['Custo_Histórico_da_Viagem']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar uma lista de índices das colunas categóricas
categorical_columns = ['Categoria_de_Localização', 'Status_de_Lealdade_do_Cliente', 'Horário_da_Reserva', 'Tipo_de_Veículo']

# Criar um transformador para aplicar codificação one-hot nas colunas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'
)

# Aplicar o pré-processamento
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Instanciar o modelo de regressão linear
modelo_regressao = LinearRegression()

# Treinar o modelo com os dados de treinamento
modelo_regressao.fit(X_train_transformed, y_train)

# Fazer previsões usando os dados de teste
previsoes = modelo_regressao.predict(X_test_transformed)

# Avaliar o desempenho do modelo
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, previsoes)
r2 = r2_score(y_test, previsoes)

print("Erro Quadrático Médio (MSE):", mse)
print("Coeficiente de Determinação (R²):", r2)


In [ ]:
# Calcular o índice de satisfação do cliente com base nas Avaliações Médias
indice_satisfacao = df['Avaliações_Médias'].mean()

print("Índice de Satisfação do Cliente com base nas Avaliações Médias:", indice_satisfacao)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Dividir os dados em conjuntos de treinamento e teste
X = df[['Número_de_Viagens_Anteriores', 'Avaliações_Médias', 'Duração_Estimada_da_Viagem', 'Custo_Histórico_da_Viagem']]
y = df['Avaliações_Médias']  # Suponha que 'Satisfação_do_Cliente' seja a variável alvo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciar o modelo de regressão linear
modelo = LinearRegression()

# Treinar o modelo com os dados de treinamento
modelo.fit(X_train, y_train)

# Fazer previsões usando os dados de teste
previsoes = modelo.predict(X_test)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, previsoes)
print("Erro médio quadrático:", mse)


In [ ]:
# Identificar características numéricas e categóricas
caracteristicas_numericas = df.select_dtypes(include=['float', 'int']).columns
caracteristicas_categoricas = df.select_dtypes(include=['object']).columns

# Lidar com valores ausentes nas características numéricas
df[caracteristicas_numericas] = df[caracteristicas_numericas].fillna(df[caracteristicas_numericas].mean())

# Detectar e lidar com valores discrepantes nas características numéricas usando IQR
for feature in caracteristicas_numericas:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - (1.5 * IQR)
    limite_superior = Q3 + (1.5 * IQR)
    df[feature] = np.where((df[feature] < limite_inferior) | (df[feature] > limite_superior),
                           df[feature].mean(), df[feature])

# Lidar com valores ausentes nas características categóricas
df[caracteristicas_categoricas] = df[caracteristicas_categoricas].fillna(df[caracteristicas_categoricas].mode().iloc[0])

# Exibir o DataFrame modificado
df.head()


In [ ]:
# Mapear os valores categóricos para valores numéricos
tipo_veiculo_mapping = {'Economy': 0, 'Standard': 1, 'Premium': 2}

# Aplicar o mapeamento à coluna 'Tipo_de_Veículo'
df['Tipo_de_Veículo'] = df['Tipo_de_Veículo'].replace(tipo_veiculo_mapping)

# Exibir o DataFrame modificado
df


In [ ]:
df.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Separar os dados em variáveis de entrada (x) e variável de saída (y)
x = np.array(df[["Número_de_Passageiros", "Número_de_Motoristas", "Tipo_de_Veículo", "Duração_Estimada_da_Viagem"]])
y = np.array(df["custo_viagem_ajustado"])

# Dividir os dados em conjuntos de treinamento e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,
                                                        y,
                                                        test_size=0.2,
                                                        random_state=42)

# Reformular y para um array 1D
y_treino = y_treino.ravel()
y_teste = y_teste.ravel()

# Treinando um modelo de regressão de floresta aleatória
modelo = RandomForestRegressor()
modelo.fit(x_treino, y_treino)


In [ ]:
vehicle_type_mapping = {
    "Premium": 1,
    "Economy": 0
}

def get_vehicle_type_numeric(vehicle_type):
    vehicle_type_numeric = vehicle_type_mapping.get(vehicle_type)
    return vehicle_type_numeric

# Predizendo usando valores de entrada do usuário
def prever_preco(numero_de_passageiros, numero_de_motoristas, tipo_de_veiculo, Duracao_Estimada_da_Viagem):
    tipo_de_veiculo_numerico = get_vehicle_type_numeric(tipo_de_veiculo)
    if tipo_de_veiculo_numerico is None:
        raise ValueError("Tipo de veículo inválido")
    
    dados_entrada = np.array([[numero_de_passageiros, numero_de_motoristas, tipo_de_veiculo_numerico, Duracao_Estimada_da_Viagem]])
    preco_previsto = modelo.predict(dados_entrada)
    return preco_previsto

# Exemplo de previsão usando valores de entrada do usuário
numero_de_passageiros_usuario = 50
numero_de_motoristas_usuario = 25
tipo_de_veiculo_usuario = "Economy"
Duracao_Estimada_da_Viagem_usuario = 30
preco_previsto = prever_preco(numero_de_passageiros_usuario, numero_de_motoristas_usuario, tipo_de_veiculo_usuario, Duracao_Estimada_da_Viagem_usuario)
print("Preço previsto:", preco_previsto)


In [ ]:
# Fazer previsões no conjunto de teste
y_pred = modelo.predict(x_teste)

# Criar um gráfico de dispersão com os valores reais vs. previstos
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_teste.flatten(),
    y=y_pred,
    mode='markers',
    name='Real vs. Previsto'
))

# Adicionar uma linha representando o caso ideal
fig.add_trace(go.Scatter(
    x=[min(y_teste.flatten()), max(y_teste.flatten())],
    y=[min(y_teste.flatten()), max(y_teste.flatten())],
    mode='lines',
    name='Ideal',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title='Valores Reais vs. Previstos',
    xaxis_title='Valores Reais',
    yaxis_title='Valores Previstos',
    showlegend=True,
)

fig.show()
